<a href="https://colab.research.google.com/github/rafiulbiswas/Ozempic/blob/main/SideEffect_Ozempic.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
%%capture
!pip install unsloth
# Also get the latest nightly Unsloth!
!pip uninstall unsloth -y && pip install --upgrade --no-cache-dir --no-deps git+https://github.com/unslothai/unsloth.git

In [ ]:
from unsloth import FastLanguageModel
import torch
max_seq_length = 2048 # Choose any! We auto support RoPE Scaling internally!
dtype = None # None for auto detection. Float16 for Tesla T4, V100, Bfloat16 for Ampere+
load_in_4bit = True # Use 4bit quantization to reduce memory usage. Can be False.

# 4bit pre quantized models we support for 4x faster downloading + no OOMs.
fourbit_models = [
    "unsloth/Meta-Llama-3.1-8B-bnb-4bit",      # Llama-3.1 15 trillion tokens model 2x faster!
    "unsloth/Meta-Llama-3.1-8B-Instruct-bnb-4bit",
    "unsloth/Meta-Llama-3.1-70B-bnb-4bit",
    "unsloth/Meta-Llama-3.1-405B-bnb-4bit",    # We also uploaded 4bit for 405b!
    "unsloth/Mistral-Nemo-Base-2407-bnb-4bit", # New Mistral 12b 2x faster!
    "unsloth/Mistral-Nemo-Instruct-2407-bnb-4bit",
    "unsloth/mistral-7b-v0.3-bnb-4bit",        # Mistral v3 2x faster!
    "unsloth/mistral-7b-instruct-v0.3-bnb-4bit",
    "unsloth/Phi-3.5-mini-instruct",           # Phi-3.5 2x faster!
    "unsloth/Phi-3-medium-4k-instruct",
    "unsloth/gemma-2-9b-bnb-4bit",
    "unsloth/gemma-2-27b-bnb-4bit",            # Gemma 2x faster!
] # More models at https://huggingface.co/unsloth

model, tokenizer = FastLanguageModel.from_pretrained(
    model_name = "unsloth/Meta-Llama-3.1-8B-Instruct-bnb-4bit",
    max_seq_length = max_seq_length,
    dtype = dtype,
    load_in_4bit = load_in_4bit,
    # token = "hf_...", # use one if using gated models like meta-llama/Llama-2-7b-hf
)

🦥 Unsloth: Will patch your computer to enable 2x faster free finetuning.
🦥 Unsloth Zoo will now patch everything to make training faster!
==((====))==  Unsloth 2024.12.8: Fast Llama patching. Transformers: 4.46.3.
   \\   /|    GPU: Tesla T4. Max memory: 14.748 GB. Platform: Linux.
O^O/ \_/ \    Torch: 2.5.1+cu121. CUDA: 7.5. CUDA Toolkit: 12.1. Triton: 3.1.0
\        /    Bfloat16 = FALSE. FA [Xformers = 0.0.28.post3. FA2 = False]
 "-____-"     Free Apache license: http://github.com/unslothai/unsloth
Unsloth: Fast downloading is enabled - ignore downloading bars which are red colored!


model.safetensors:   0%|          | 0.00/5.70G [00:00<?, ?B/s]

generation_config.json:   0%|          | 0.00/234 [00:00<?, ?B/s]

tokenizer_config.json:   0%|          | 0.00/55.4k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/9.09M [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/340 [00:00<?, ?B/s]

In [ ]:
model = FastLanguageModel.get_peft_model(
    model,
    r = 16, # Choose any number > 0 ! Suggested 8, 16, 32, 64, 128
    target_modules = ["q_proj", "k_proj", "v_proj", "o_proj",
                      "gate_proj", "up_proj", "down_proj",],
    lora_alpha = 16,
    lora_dropout = 0, # Supports any, but = 0 is optimized
    bias = "none",    # Supports any, but = "none" is optimized
    # [NEW] "unsloth" uses 30% less VRAM, fits 2x larger batch sizes!
    use_gradient_checkpointing = "unsloth", # True or "unsloth" for very long context
    random_state = 3407,
    use_rslora = False,  # We support rank stabilized LoRA
    loftq_config = None, # And LoftQ
)

Unsloth 2024.12.8 patched 32 layers with 32 QKV layers, 32 O layers and 32 MLP layers.


In [ ]:
from datasets import Dataset
import pandas as pd

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [ ]:
import pandas as pd
import json

# Function to read JSON file and convert to DataFrame
def json_to_dataframe(json_file_path):
    data_list = []
    with open(json_file_path, 'r', encoding='utf-8') as file:
        for line in file:
            try:
                data = json.loads(line)
                data_list.append(data)
            except json.JSONDecodeError as e:
                print(f"Error decoding JSON: {e}")
    df = pd.json_normalize(data_list)
    return df

# Example usage
json_file_path = '/content/drive/MyDrive/Colab Notebooks/273_stigma_community.jsonl'  # Replace with your JSON file path
df = json_to_dataframe(json_file_path)



In [ ]:
df.head(2)

,instruction,output,text
0,Classify the text into one of the following ca...,none,popular weight-loss drugs like wegovy may rais...
1,Categorize text based on sentiment and usage c...,none,'ozempic n maybe gastric sleeve .. either way ...


In [ ]:

# Assuming the CSV contains 'input' and 'output' columns
# Define the Alpaca-style prompt format
alpaca_prompt = """You're a social media expert in analyzing public stigma and community reactions towards Ozempic/Semaglutide medicine. Your task is to perform according to instructions.

### Instruction:
{}

### Input:
{}

### Response:
{}"""

# Define EOS_TOKEN (ensure you have a tokenizer or define it as needed)
EOS_TOKEN = "<|endoftext|>"  # Adjust this if necessary, depending on your tokenizer

# Create a list to store the formatted text
texts = []

# Loop over the DataFrame rows to format the text
for _, row in df.iterrows():
    instruction = row["instruction"]
    input_text = row["text"]
    output_text = row["output"]
    text = alpaca_prompt.format(instruction, input_text, output_text) + EOS_TOKEN
    texts.append(text)

# Create a new DataFrame with the formatted text
new_df = pd.DataFrame({"text": texts})
# Convert the DataFrame to a Hugging Face Dataset
dataset = Dataset.from_pandas(new_df)
# Print the new DataFrame
print(new_df)


                                                  text
0    You're a social media expert in analyzing publ...
1    You're a social media expert in analyzing publ...
2    You're a social media expert in analyzing publ...
3    You're a social media expert in analyzing publ...
4    You're a social media expert in analyzing publ...
..                                                 ...
268  You're a social media expert in analyzing publ...
269  You're a social media expert in analyzing publ...
270  You're a social media expert in analyzing publ...
271  You're a social media expert in analyzing publ...
272  You're a social media expert in analyzing publ...

[273 rows x 1 columns]


In [ ]:
dataset['text'][67]

"You're a social media expert in analyzing public stigma and community reactions towards Ozempic/Semaglutide medicine. Your task is to perform according to instructions.\n\n### Instruction:\nCategorize text based on sentiment and usage context: a. Stigma for non-medical reasons, b.Cosmetic versus medical use,c. Supportive, d. Critics, e. None\n\n### Input:\nozempic protects kidneys, boosts survival in diabetes patients with chronic kidney disease\n\n### Response:\nnone<|endoftext|>"

In [ ]:
from trl import SFTTrainer
from transformers import TrainingArguments
from unsloth import is_bfloat16_supported
# Now, proceed to define the trainer with the processed dataset
trainer = SFTTrainer(
    model=model,
    tokenizer=tokenizer,
    train_dataset=dataset,
    dataset_text_field="text",
    max_seq_length=max_seq_length,
    dataset_num_proc=2,
    packing=False,  # Can make training 5x faster for short sequences.
    args=TrainingArguments(
        per_device_train_batch_size=2,
        gradient_accumulation_steps=4,
        warmup_steps=5,
        max_steps=35,
        learning_rate=2e-4,
        fp16=not is_bfloat16_supported(),
        bf16=is_bfloat16_supported(),
        logging_steps=1,
        optim="adamw_8bit",
        weight_decay=0.01,
        lr_scheduler_type="linear",
        seed=3407,
        output_dir="outputs",
    ),
)

Map (num_proc=2):   0%|          | 0/273 [00:00<?, ? examples/s]

max_steps is given, it will override any value given in num_train_epochs


In [ ]:
trainer_stats = trainer.train()

==((====))==  Unsloth - 2x faster free finetuning | Num GPUs = 1
   \\   /|    Num examples = 273 | Num Epochs = 2
O^O/ \_/ \    Batch size per device = 2 | Gradient Accumulation steps = 4
\        /    Total batch size = 8 | Total steps = 35
 "-____-"     Number of trainable parameters = 41,943,040
wandb: WARNING The `run_name` is currently set to the same value as `TrainingArguments.output_dir`. If this was not intended, please specify a different run name by setting the `TrainingArguments.run_name` parameter.
wandb: Using wandb-core as the SDK backend.  Please refer to https://wandb.me/wandb-core for more information.


<IPython.core.display.Javascript object>

wandb: Logging into wandb.ai. (Learn how to deploy a W&B server locally: https://wandb.me/wandb-server)
wandb: You can find your API key in your browser here: https://wandb.ai/authorize
wandb: Paste an API key from your profile and hit enter, or press ctrl+c to quit:

 ··········


wandb: Appending key for api.wandb.ai to your netrc file: /root/.netrc


Step,Training Loss
1,2.940600
2,3.108600
3,2.913800
4,2.656400
5,2.528300
6,2.433000
7,2.113400
8,1.930100
9,1.877300
10,1.630000


In [ ]:
import json
import pandas as pd
from transformers import AutoTokenizer, AutoModelForCausalLM

# Load the tokenizer and model
# Replace 'your-model-name' with the specific model name
# tokenizer = AutoTokenizer.from_pretrained("your-model-name")
# model = AutoModelForCausalLM.from_pretrained("your-model-name").to("cuda")
# Enable faster inference (assuming `FastLanguageModel` is part of your setup)
FastLanguageModel.for_inference(model)  # Enable 2x faster inference (if supported by your setup)

# Read the JSONL file
jsonl_file = '/content/drive/MyDrive/Colab Notebooks/train_reddit_stigma_3rd.jsonl'

# Load the JSONL data
data = []
with open(jsonl_file, 'r', encoding='utf-8') as f:
    for line in f:
        data.append(json.loads(line))

# Convert to a DataFrame
df = pd.DataFrame(data)

# Set chunk size
chunk_size = 500
num_chunks = len(df) // chunk_size + (1 if len(df) % chunk_size != 0 else 0)

# Process the data in chunks
for chunk_idx in range(num_chunks):
    start_idx = chunk_idx * chunk_size
    end_idx = min((chunk_idx + 1) * chunk_size, len(df))
    chunk = df.iloc[start_idx:end_idx].copy()

    # Initialize an empty list to store responses
    responses = []

    # Loop through the chunk and generate responses
    for instruction, input_text in zip(chunk['instruction'], chunk['text']):
        # Prepare the input prompt
        alpaca_prompt = f"""
        You're a social media expert in analyzing public stigma of Ozempic medication.
        Classify the text into one or multiple categories of public stigma and reactions toward ozempic.

        ### Instruction:
        {instruction}

        ### Input:
        {text}

        ### Response:
        """

        # Tokenize the inputzxx
        inputs = tokenizer([alpaca_prompt], return_tensors="pt", padding=True, truncation=True).to("cuda")

        # Generate the response
        output = model.generate(**inputs, max_new_tokens=10)  # Adjust max_new_tokens as needed

        # Decode the output
        response = tokenizer.decode(output[0], skip_special_tokens=True).strip()

        # Extract the text after "### Response:" if it exists
        if "### Response:" in response:
            response = response.split("### Response:")[-1].strip()

        # Append the cleaned response to the list
        responses.append(response)

    # Add the responses to the chunk
    chunk['stigma_response'] = responses

    # Append to the output CSV file
    if chunk_idx == 0:
        chunk.to_csv("/content/drive/MyDrive/Colab Notebooks/train_reddit_stigma_3rd.csv", index=False, mode='w')  # Write mode for the first chunk
    else:
        chunk.to_csv("/content/drive/MyDrive/Colab Notebooks/train_reddit_stigma_3rd.csv", index=False, mode='a', header=False)  # Append mode for subsequent chunks

print("Processing completed! All chunks have been processed and saved.")


In [ ]:
df1.head(2)

,id,date,text,sentiment_label,sentiment_negative,sentiment_neutral,sentiment_positive
0,14e2fgi,2023-06-20 05:52:08,This is going to be a long one. I'll start wit...,Negative,0.737981,0.240828,0.021191
1,r916km,2021-12-04 22:45:21,My doctor decided to start me on Wegovy. He ga...,Neutral,0.057079,0.863082,0.079838


In [ ]:
df2=df1.sample(frac=0.0005)

In [ ]:
df2.shape[0]

10

In [ ]:
import pandas as pd
from transformers import AutoTokenizer, AutoModelForCausalLM

# Load the tokenizer and model
# Replace 'your-model-name' with the specific model name
# tokenizer = AutoTokenizer.from_pretrained("your-model-name")
# model = AutoModelForCausalLM.from_pretrained("your-model-name").to("cuda")

# Define the instruction
instruction = """Analyze the provided text and determine if it discusses side effects. Classify the text into one or more of the following categories:
1. Types of Side Effects: Identify the specific side effects mentioned, such as nausea, vomiting, fatigue, or other physical or psychological effects.
2. Severity and Frequency": Assess whether the side effects are perceived as mild, moderate, or severe and how frequently they are mentioned in the text.
3. Impact on Daily Life: Determine if the reported side effects significantly affect the users' daily routines or quality of life.
4. Impact with Time: Evaluate whether the side effects are improving with continued use."""

# Initialize an empty list to store responses
responses = []

# Loop through the DataFrame and generate responses
for input_text in df1['text']:
    # Prepare the input prompt
    alpaca_prompt = f"""
    You're a social media expert in analyzing public attitudes towards side effects of Ozempic.
    Classify the text into one or multiple categories of side effect discussed.

    ### Instruction:
    {instruction}

    ### Input:
    {input_text}

    ### Response:
    """

    # Tokenize the input
    inputs = tokenizer([alpaca_prompt], return_tensors="pt", padding=True, truncation=True).to("cuda")

    # Generate the response
    output = model.generate(**inputs, max_new_tokens=10)  # Adjust max_new_tokens as needed

    # Decode the output
    response = tokenizer.decode(output[0], skip_special_tokens=True).strip()

    # Extract the text after "### Response:" if it exists
    if "### Response:" in response:
        response = response.split("### Response:")[-1].strip()

    # Append the cleaned response to the list
    responses.append(response)

# Add the responses to the DataFrame
df1['sideeffect_response'] = responses

# Save the DataFrame with responses
df1.to_csv("df_train.csv", index=False)


In [ ]:
df2.head(10)

,id,date,text,sentiment_label,sentiment_negative,sentiment_neutral,sentiment_positive,sideeffect_response
13249,wj3mvl,2022-08-08 08:35:42,Some people may be aware that the drug “Seroqu...,Negative,0.541857,0.440178,0.017965,none<|endoftext|>none<
2339,15knwzj,2023-08-07 15:28:20,I searched this group but couldn’t find any an...,Neutral,0.344182,0.619878,0.035940,none<|endoftext|>none<
11590,14k9s83,2023-06-27 10:22:19,\n\n**I have been overweight since I was 15 y...,Negative,0.546266,0.412692,0.041042,None<|endoftext|>### Response
18262,12y3boh,2023-04-25 01:11:25,I just started wegovy and can already tell thi...,Positive,0.048867,0.285890,0.665243,None <br />#wegovy #weight
6913,tpmjze,2022-03-27 15:44:06,Has anyone else had such a big stretch between...,Negative,0.478555,0.439475,0.081970,None<|endoftext|>il|
499,ruliek,2022-01-02 22:39:23,My insurance (UMR) won’t approve Ozempic and m...,Negative,0.589602,0.347717,0.062681,none<|endoftext|>none<
10731,12b4z23,2023-04-04 01:26:33,My ozempic side effects are SO bad. I’ve been ...,Negative,0.887346,0.097622,0.015032,none\n ### Response Type:\n none
15697,117o6iz,2023-02-20 23:32:42,I started Ozempic on Friday at 0.25 mg and sin...,Neutral,0.262381,0.550453,0.187166,none<|endoftext|> none<
336,1fpsadj,09-26-2024 09:40:35,I have been on ozempic 2 months and began havi...,Negative,0.889937,0.098425,0.011638,severity and frequency<|endoftext|>
10226,18hqjey,2023-12-13 20:38:13,Last Wednesday I took my 1st .25 mg dose of Oz...,Positive,0.017800,0.095418,0.886782,"types of side effects, severity and frequency,..."
